In [2]:
import os
import json
import re
import shutil
input_folder = "encoded"
output_folder = "done"
import torch
import torch.nn.functional as F


In [3]:
# Use list comprehension to create a list of file paths
verilog_files = [os.path.join(input_folder, filename) for filename in os.listdir(input_folder) if filename.endswith('.txt')]

# Print the list of file paths
print(len(verilog_files))
print(type(verilog_files[0]))
print(verilog_files[0])


298
<class 'str'>
encoded\adder11.txt


In [4]:
# ground_truth_labels = [0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15] # no bcd, no seg
ground_truth_labels = [0,1,2,3,4,5,6,7,8,9,10,11,12,13,14] # no xor 
# ground_truth_labels = [0,1,2,3,4,5,6,7,8] # no gates
labels_torch = torch.tensor(ground_truth_labels)

# One-hot encode the labels
# one_hot_labels = F.one_hot(labels_torch, num_classes=16)
# one_hot_labels = F.one_hot(labels_torch, num_classes=9)
one_hot_labels = F.one_hot(labels_torch, num_classes=15)

print(one_hot_labels[0])

tensor([1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0])


In [5]:

labels_dict = dict()
def label_verilog_file(file_name):
    # label_mapping = {
    #     'adder': 0, 'ALU': 1, 'comparator': 2, 'decoder': 3,
    #     'encoder': 4, 'mult': 5, 'mux': 6, 'pe': 7, 'sub': 8
    # }
    label_mapping = {
        'adder': 0, 'ALU': 1, 'comparator': 2, 'decoder': 3,
        'encoder': 4, 'mult': 5, 'mux': 6, 'pe': 7, 'sub': 8, 'and': 9, 'or': 10, 'not': 11, 'nand': 12, 'nor': 13, 'xnor': 14
    }
    
    pattern = r"([a-zA-Z]+)(\d+)?"
    match = re.match(pattern, file_name)
    if match:
        base_name = match.group(1)
        if base_name in label_mapping:
            labels_dict[base_name] = labels_dict.get(base_name, 0) + 1
            return one_hot_labels[label_mapping[base_name]].tolist()
        
    return None


def add_label_to_verilog_file(input_file_path, output_folder):
    if input_file_path.endswith('.txt'):
        with open(input_file_path, "r") as file:
            loaded_data = json.load(file)
            label = label_verilog_file(os.path.basename(input_file_path))
            if label is not None and [label] not in loaded_data:  # Check if label already exists
                loaded_data.append([label])  # Add label as the third list
                output_file_path = os.path.join(output_folder, os.path.basename(input_file_path))
                with open(output_file_path, "w") as output_file:
                    json.dump(loaded_data, output_file)
                return True
    return False



# Create the output folder if it doesn't exist
if not os.path.exists(output_folder):
    os.makedirs(output_folder)

# Process files in the input folder
for file_name in os.listdir(input_folder):
    file_path = os.path.join(input_folder, file_name)
    if os.path.isfile(file_path):
        if not add_label_to_verilog_file(file_path, output_folder):
            print(f"Failed to label: {file_path}")



In [6]:
failed_files = []

def get_files_in_folder(input_folder):
    file_list = []
    for file_name in os.listdir(input_folder):
        file_path = os.path.join(input_folder, file_name)
        if os.path.isfile(file_path):
            file_list.append(file_path)
    return file_list

# Replace 'input_folder' with the path to the folder you want to read files from
verilog_files = get_files_in_folder(output_folder)
print(len(verilog_files))
print(failed_files)
print(verilog_files)

# for v in verilog_files:
#     with open(v, "r") as file:
#         loaded_data = json.load(file)
#         # print(loaded_data[0])
        
#         for node in loaded_data[0]:
#             if len(node) != 4:
#                 failed_files.append(v)
#                 break
            
#how to save list in a txt file


298
[]
['done\\adder11.txt', 'done\\adder12.txt', 'done\\adder13.txt', 'done\\adder14.txt', 'done\\adder15.txt', 'done\\adder16.txt', 'done\\adder17.txt', 'done\\adder18.txt', 'done\\adder19.txt', 'done\\adder2.txt', 'done\\adder20.txt', 'done\\adder4.txt', 'done\\adder5.txt', 'done\\adder6.txt', 'done\\adder8.txt', 'done\\ALU10.txt', 'done\\ALU13.txt', 'done\\ALU14.txt', 'done\\ALU15.txt', 'done\\ALU2.txt', 'done\\ALU6.txt', 'done\\ALU7.txt', 'done\\ALU8.txt', 'done\\ALU9.txt', 'done\\and1.txt', 'done\\and10.txt', 'done\\and12.txt', 'done\\and13.txt', 'done\\and14.txt', 'done\\and15.txt', 'done\\and16.txt', 'done\\and17.txt', 'done\\and18.txt', 'done\\and19.txt', 'done\\and2.txt', 'done\\and20.txt', 'done\\and21.txt', 'done\\and23.txt', 'done\\and25.txt', 'done\\and26.txt', 'done\\and27.txt', 'done\\and28.txt', 'done\\and29.txt', 'done\\and3.txt', 'done\\and30.txt', 'done\\and6.txt', 'done\\and7.txt', 'done\\and8.txt', 'done\\and9.txt', 'done\\comparator1.txt', 'done\\comparator13.txt

# reduce classes: 
    -Decoder
    -Encoder
    -Mux

removed seg and bcd classes

In [8]:
# print(labels_dict)
# data_str_keys = {str(key): value for key, value in labels_dict.items()}
with open("labels_dict.txt", "w") as file:
    json.dump(labels_dict, file)

labels_dict

{'adder': 15,
 'ALU': 9,
 'and': 25,
 'comparator': 28,
 'decoder': 26,
 'encoder': 23,
 'mult': 25,
 'mux': 26,
 'nand': 20,
 'nor': 16,
 'not': 11,
 'or': 24,
 'pe': 17,
 'sub': 10,
 'xnor': 23}

In [ ]:
def get_files_in_folder(input_folder):
    file_list = []
    for file_name in os.listdir(input_folder):
        file_path = os.path.join(input_folder, file_name)
        if os.path.isfile(file_path):
            file_list.append(file_path)
    return file_list

In [ ]:
def preprocessing(folder_path):
    
    file_list = get_files_in_folder(folder_path)
    
    return file_list